## **Laboratorio 5**
- Sofía García - 22210
- Joaquín Campos - 22155
- Julio García Salas - 22076

## **Inciso 1 y 2**

In [1]:
import pandas as pd
import numpy as np
import os
import re

print("Versions -> pandas:", pd.__version__)

# ---- 1) Rutas candidatas (usa la que tengas local) ----
TRAIN_CANDIDATES = ["train.csv", "./train.csv", "/mnt/data/train.csv"]
TEST_CANDIDATES  = ["test.csv", "./test.csv", "/mnt/data/test.csv"]

def load_first_available(paths):
    last_err = None
    for p in paths:
        try:
            df = pd.read_csv(p)
            print(f"✔️  Cargado: {p}  -> shape={df.shape}")
            return df, p
        except Exception as e:
            last_err = e
    raise FileNotFoundError(f"No se pudo cargar desde {paths}. Último error: {last_err}")

train_df, train_path = load_first_available(TRAIN_CANDIDATES)

# test.csv es opcional para este paso
try:
    test_df, test_path = load_first_available(TEST_CANDIDATES)
except Exception:
    test_df, test_path = None, None
    print("ℹ️  test.csv no encontrado (no es obligatorio para este inciso).")

# ---- 2) Intento de detección de columnas clave ----
def guess_text_col(df):
    candidates = ["text", "tweet", "content", "message", "Text", "Tweet"]
    for c in candidates:
        if c in df.columns:
            return c
    # fallback: columna object con mayor longitud media
    obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    if not obj_cols:
        return None
    def avg_len(s):
        try:
            return s.dropna().astype(str).str.len().mean()
        except Exception:
            return -1
    best = max(obj_cols, key=lambda c: avg_len(df[c]))
    return best

def guess_target_col(df):
    candidates = ["target", "label", "is_disaster", "Target", "Label"]
    for c in candidates:
        if c in df.columns:
            return c
    return None

text_col   = guess_text_col(train_df)
target_col = guess_target_col(train_df)

print(f"🧭 Columna de texto detectada: {text_col!r}")
print(f"🧭 Columna de etiqueta detectada: {target_col!r}")

# ---- 3) Descripción general del train ----
print("\n=== DESCRIPCIÓN GENERAL (train) ===")
print("Shape:", train_df.shape)
print("\nColumnas y tipos:\n", train_df.dtypes)

print("\nValores nulos por columna:")
print(train_df.isna().sum())

# Duplicados (por fila completa y por texto)
dup_rows = train_df.duplicated().sum()
print(f"\nFilas duplicadas (todas las columnas): {dup_rows}")

if text_col is not None:
    dup_texts = train_df.duplicated(subset=[text_col]).sum()
    print(f"Filas con {text_col!r} duplicado: {dup_texts}")

# Distribución de clases (si hay etiqueta)
if target_col is not None:
    print("\nDistribución de clases:")
    print(train_df[target_col].value_counts(dropna=False))
    print("\nProporción de clases:")
    print(train_df[target_col].value_counts(normalize=True, dropna=False).round(3))

# Longitud de textos
if text_col is not None:
    tmp = train_df[text_col].astype(str)
    char_len = tmp.str.len()
    token_len = tmp.str.split().apply(len)

    print("\n=== Estadísticas de longitud de texto ===")
    print("Caracteres -> mean:", round(char_len.mean(), 1),
          "| std:", round(char_len.std(), 1),
          "| min:", int(char_len.min()),
          "| p50:", int(char_len.median()),
          "| p95:", int(char_len.quantile(0.95)),
          "| max:", int(char_len.max()))
    print("Tokens     -> mean:", round(token_len.mean(), 1),
          "| std:", round(token_len.std(), 1),
          "| min:", int(token_len.min()),
          "| p50:", int(token_len.median()),
          "| p95:", int(token_len.quantile(0.95)),
          "| max:", int(token_len.max()))

# Muestra rápida de ejemplos por clase (si hay etiqueta)
def sample_by_class(df, label_col, k=3):
    out = {}
    if label_col is None:
        return out
    for cls, group in df.groupby(label_col):
        out[cls] = group.sample(n=min(k, len(group)), random_state=42)
    return out

print("\n=== MUESTRAS DE TEXTO (por clase si aplica) ===")
if target_col is not None and text_col is not None:
    samples = sample_by_class(train_df, target_col, k=3)
    for cls, df_s in samples.items():
        print(f"\n>> Clase = {cls} (muestras):")
        for i, row in df_s.iterrows():
            txt = str(row[text_col])
            txt = re.sub(r"\s+", " ", txt).strip()
            print("-", txt[:200] + ("..." if len(txt) > 200 else ""))
else:
    # si no hay target, solo mostramos algunas filas
    print(train_df.head(5))

# ---- 4) (Opcional) vista previa del test ----
if test_df is not None:
    print("\n=== PREVIEW test.csv ===")
    print("Shape:", test_df.shape)
    print("Columnas:", list(test_df.columns)[:10])
    if text_col and text_col in test_df.columns:
        print("\nEjemplos test:")
        for t in test_df[text_col].astype(str).head(3):
            t = re.sub(r"\s+", " ", t).strip()
            print("-", t[:200] + ("..." if len(t) > 200 else ""))


Versions -> pandas: 2.2.3
✔️  Cargado: train.csv  -> shape=(7613, 5)
✔️  Cargado: test.csv  -> shape=(3263, 4)
🧭 Columna de texto detectada: 'text'
🧭 Columna de etiqueta detectada: 'target'

=== DESCRIPCIÓN GENERAL (train) ===
Shape: (7613, 5)

Columnas y tipos:
 id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

Valores nulos por columna:
id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

Filas duplicadas (todas las columnas): 0
Filas con 'text' duplicado: 110

Distribución de clases:
target
0    4342
1    3271
Name: count, dtype: int64

Proporción de clases:
target
0    0.57
1    0.43
Name: proportion, dtype: float64

=== Estadísticas de longitud de texto ===
Caracteres -> mean: 101.0 | std: 33.8 | min: 7 | p50: 107 | p95: 140 | max: 157
Tokens     -> mean: 14.9 | std: 5.7 | min: 1 | p50: 15 | p95: 24 | max: 31

=== MUESTRAS DE TEXTO (por clase si aplica) ===

>> Clase = 0 (mues

In [3]:
# Cell 2 — Preprocesamiento detallado (inciso 3) — versión corregida

import re, html, unicodedata, string
import pandas as pd
import numpy as np

# --- Asegurarnos de tener train_df, text_col y target_col (del Cell 1) ---
if "train_df" not in globals():
    train_df = pd.read_csv("train.csv")
if "text_col" not in globals():
    text_col = "text" if "text" in train_df.columns else train_df.select_dtypes("object").columns[0]
if "target_col" not in globals():
    target_col = "target" if "target" in train_df.columns else None

print(f"Usando columnas -> texto: {text_col!r} | etiqueta: {target_col!r}")

# --- Stopwords en inglés (evitamos dependencias de descarga) ---
try:
    from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
    STOPWORDS = set(ENGLISH_STOP_WORDS)
except Exception:
    # Fallback mínimo si sklearn no está disponible
    STOPWORDS = set("""
a about above after again against all am an and any are as at be because been before being below between both
but by can did do does doing down during each few for from further had has have having he she'd he'll she's her
here hers herself him himself his how i i'd i'll i'm i've if in into is it it's its itself let me more most my
myself no nor not of off on once only or other our ours ourselves out over own same she should so some such
than that that's the their theirs them themselves then there there's these they they'd they'll they're they've
this those through to too under until up very was we we'd we'll we're we've were what what's when when's where
where's which while who who's whom why why's with you you'd you'll you're you've your yours yourself yourselves
""".split())

# Ajustes de stopwords específicos de tweets
STOPWORDS |= {"rt", "amp", "https", "http", "co", "t"}  # &amp; y tokens comunes de URLs acortadas

# --- Compilar patrones regex reutilizables ---
URL_RE        = re.compile(r"(https?://\S+|www\.\S+)", flags=re.IGNORECASE)
MENTION_RE    = re.compile(r"@\w+")
HASHTAG_RE    = re.compile(r"#(\w+)")
HTML_ENT_RE   = re.compile(r"&[a-z]+;")
EMOTICON_RE   = re.compile(r"(?::|;|=|8)(?:-|'|o)?(?:\)|\(|D|P|/|\\|\||\]|\[)", flags=re.IGNORECASE)

# Rango amplio de emojis (unicode)
EMOJI_RE = re.compile(
    "[" +
    "\U0001F300-\U0001F5FF" +  # pictos y símbolos
    "\U0001F600-\U0001F64F" +  # emoticonos
    "\U0001F680-\U0001F6FF" +  # transporte
    "\U0001F700-\U0001F77F" +
    "\U0001F780-\U0001F7FF" +
    "\U0001F800-\U0001F8FF" +
    "\U0001F900-\U0001F9FF" +
    "\U0001FA00-\U0001FA6F" +
    "\U0001FA70-\U0001FAFF" +
    "\U00002700-\U000027BF" +  # dingbats
    "\U00002600-\U000026FF" +  # misceláneo
    "]+",
    flags=re.UNICODE
)

PUNCT_TABLE = str.maketrans("", "", string.punctuation)  # elimina puntuación ASCII

def normalize_unicode(text: str) -> str:
    # Normaliza a NFKC para unificar formas y símbolos "raros"
    return unicodedata.normalize("NFKC", text)

def preprocess_text(s: str, keep_number_token="911"):
    orig = str(s)

    # Métricas de sucio
    urls_found     = len(URL_RE.findall(orig))
    mentions_found = len(MENTION_RE.findall(orig))
    hashtags_found = len(HASHTAG_RE.findall(orig))
    emojis_found   = len(EMOJI_RE.findall(orig)) + len(EMOTICON_RE.findall(orig))

    # 1) Minúsculas + desenmascarar HTML (&amp; -> &)
    t = html.unescape(orig).lower()

    # 2) Normalización unicode
    t = normalize_unicode(t)

    # 3) Eliminar URLs completas
    t = URL_RE.sub(" ", t)

    # 4) Eliminar @menciones
    t = MENTION_RE.sub(" ", t)

    # 5) Mantener el texto del hashtag, quitando el '#'
    #    "#wildfire" -> "wildfire"
    t = HASHTAG_RE.sub(r"\1", t)

    # 6) Eliminar emojis y emoticones
    t = EMOJI_RE.sub(" ", t)
    t = EMOTICON_RE.sub(" ", t)

    # 7) Eliminar puntuación ASCII (quita apóstrofes, comas, etc.)
    t = t.translate(PUNCT_TABLE)

    # 8) Quitar residuos de entidades HTML restantes
    t = HTML_ENT_RE.sub(" ", t)

    # 9) Tokenizar por espacios
    tokens = [tok for tok in t.split() if tok]

    # 10) Quitar stopwords
    tokens = [tok for tok in tokens if tok not in STOPWORDS]

    # 11) Manejo de números:
    #     - Eliminamos tokens NUMÉRICOS EXCEPTO "911" (mantener porque puede ser relevante en desastres)
    #     - Para tokens alfanuméricos, conservamos tal cual (e.g., "h2o", "m5.0")
    cleaned_tokens = []
    for tok in tokens:
        if tok.isdigit():
            if keep_number_token and tok == keep_number_token:
                cleaned_tokens.append(tok)
            # si es otro número puro, lo omitimos
        else:
            cleaned_tokens.append(tok)

    # 12) Reconstrucción
    clean_text = " ".join(cleaned_tokens)

    # Métricas post
    return {
        "clean_text": clean_text,
        "tokens": cleaned_tokens,
        "n_urls": urls_found,
        "n_mentions": mentions_found,
        "n_hashtags": hashtags_found,
        "n_emojis": emojis_found,
        "orig_len_chars": len(orig),
        "orig_len_tokens": len(orig.split()),
        "clean_len_tokens": len(cleaned_tokens),
    }

# --- Aplicar preprocesamiento ---
proc = train_df[text_col].astype(str).apply(preprocess_text)

train_df["text_clean"]   = proc.apply(lambda x: x["clean_text"])
train_df["tokens_clean"] = proc.apply(lambda x: x["tokens"])

# Métricas agregadas del proceso
report = pd.DataFrame({
    "urls":         proc.apply(lambda x: x["n_urls"]),
    "mentions":     proc.apply(lambda x: x["n_mentions"]),
    "hashtags":     proc.apply(lambda x: x["n_hashtags"]),
    "emojis":       proc.apply(lambda x: x["n_emojis"]),
    "orig_chars":   proc.apply(lambda x: x["orig_len_chars"]),
    "orig_tokens":  proc.apply(lambda x: x["orig_len_tokens"]),
    "clean_tokens": proc.apply(lambda x: x["clean_len_tokens"]),
})

# --- Reporte en consola ---
print("\n=== REPORTE DE PREPROCESAMIENTO (train) ===")
print(f"Filas procesadas: {len(train_df)}")
print("Totales removidos/detectados:")
print(" • URLs:     ", int(report["urls"].sum()))
print(" • Menciones:", int(report["mentions"].sum()))
print(" • Hashtags: ", int(report["hashtags"].sum()), "(se conservó la palabra, sin '#')")
print(" • Emojis:   ", int(report["emojis"].sum()))

print("\nLongitud de tokens (antes vs después):")
print(" • Tokens (orig)  -> mean:", round(report["orig_tokens"].mean(),1),
      "| p50:", int(report["orig_tokens"].median()),
      "| p95:", int(report["orig_tokens"].quantile(0.95)))
print(" • Tokens (clean) -> mean:", round(report["clean_tokens"].mean(),1),
      "| p50:", int(report["clean_tokens"].median()),
      "| p95:", int(report["clean_tokens"].quantile(0.95)))

# Distribución por clase (si hay etiqueta) — usando longitud de tokens limpios
if target_col is not None and target_col in train_df.columns:
    train_df["tok_len"] = train_df["tokens_clean"].apply(len)
    by_cls = train_df.groupby(target_col)["tok_len"].agg(["mean","median","max","min"])
    print("\nTamaño de texto limpio por clase (número de tokens):")
    print(by_cls.round(2))

# --- Ejemplos antes/después por clase (para documentación del inciso 3) ---
def show_examples(df, k=3):
    groups = df.groupby(target_col) if (target_col is not None and target_col in df.columns) else [(None, df)]
    for cls_val, g in groups:
        print(f"\n>> Ejemplos clase={cls_val}:")
        samp = g.sample(n=min(k, len(g)), random_state=42)
        for _, r in samp.iterrows():
            raw = str(r[text_col]).strip().replace("\n", " ")
            clean = str(r["text_clean"]).strip()
            print("- RAW  :", raw[:160] + ("..." if len(raw) > 160 else ""))
            print("  CLEAN:", clean[:160] + ("..." if len(clean) > 160 else ""))

show_examples(train_df, k=3)

# --- Chequeo rápido de vacíos tras limpieza (p.ej. posts que quedan sin tokens) ---
empties = train_df["text_clean"].fillna("").str.strip().eq("")
n_empties = int(empties.sum())
print(f"\nPublicaciones que quedaron vacías tras limpieza: {n_empties}")

# --- Vista previa final ---
print("\n=== Vista previa columnas limpias ===")
print(train_df[[text_col, "text_clean"]].head(5))

# Nota: NO eliminamos duplicados aquí para no alterar el dataset base;
# si lo necesitas para el modelo, podemos hacerlo en el inciso del modelado.


Usando columnas -> texto: 'text' | etiqueta: 'target'

=== REPORTE DE PREPROCESAMIENTO (train) ===
Filas procesadas: 7613
Totales removidos/detectados:
 • URLs:      4723
 • Menciones: 2715
 • Hashtags:  3330 (se conservó la palabra, sin '#')
 • Emojis:    4888

Longitud de tokens (antes vs después):
 • Tokens (orig)  -> mean: 14.9 | p50: 15 | p95: 24
 • Tokens (clean) -> mean: 8.2 | p50: 8 | p95: 14

Tamaño de texto limpio por clase (número de tokens):
        mean  median  max  min
target                        
0       7.77     8.0   20    0
1       8.77     9.0   21    0

>> Ejemplos clase=0:
- RAW  : Everyday is a near death fatality for me on the road. Thank god is on my side.??
  CLEAN: everyday near death fatality road thank god
- RAW  : #Lifestyle Û÷It makes me sickÛª: Baby clothes deemed a Û÷hazardÛª http://t.co/0XrfVidxA2 http://t.co/oIHwgEZDCk
  CLEAN: lifestyle û÷it makes sickûa baby clothes deemed û÷hazardûa
- RAW  : @Lenn_Len Probably. We are inundated with them 


# Inciso 3 — Análisis del preprocesamiento (train.csv)

**Columnas usadas** → texto: `text` | etiqueta: `target`  
**Filas procesadas**: 7,613

---

## 1) Qué se limpió/eliminó

- **URLs** detectadas: **4,723**  
- **@Menciones**: **2,715**  
- **#Hashtags**: **3,330** *(se conserva la palabra, sin ‘#’)*  
- **Emojis/Emoticones**: **4,888**

> Comentario: El dataset tiene una **alta carga social** (muchas URLs, menciones y emojis). Limpiar estos elementos ayuda a reducir ruido y sesgos hacia la forma de escritura en Twitter sin eliminar señales semánticas clave (p. ej., *wildfire*, *evacuation*, *floods*).

---

## 2) Longitud de texto (antes vs después)

- **Tokens (original)** → *mean*: **14.9**, *p50*: **15**, *p95*: **24**  
- **Tokens (limpio)** → *mean*: **8.2**, *p50*: **8**, *p95*: **14**

> Interpretación: tras la limpieza, el texto queda **~45% más corto** en promedio. Esto sugiere que gran parte del contenido eran conectores, signos, URLs y marcas sociales. La reducción es esperable y suele **mejorar la densidad semántica** por token.

---

## 3) Longitud por clase (tokens limpios)

| target | mean | median | max | min |
|:------:|-----:|-------:|----:|----:|
|   0    | 7.77 |  8.00  | 20  |  0  |
|   1    | 8.77 |  9.00  | 21  |  0  |

> Observación: Los tweets **etiquetados como desastre (1)** son, en promedio, **~1 token más largos**. No es una diferencia enorme, pero puede indicar que los tweets de desastres incluyen **más contexto** (p. ej., lugar + evento + afectación).

---

## 4) Ejemplos ilustrativos (antes → después)

**Clase 0 (no desastre):**
- *everyday is a near death fatality…* → `everyday near death fatality road thank god`  
  - Riesgo de **falsos positivos**: palabras de alto “drama” (*death*, *fatality*) en contextos figurados o hiperbolizados.
- *#Lifestyle… hazard* → `lifestyle … makes sick … baby clothes deemed … hazard`  
  - Muestra ruido de codificación (ver “mojibake” abajo).
- *@Lenn_Len… inundated* → `probably inundated years`  
  - *inundated* aquí no describe una **inundación real**, sino “abrumados” → **ambigüedad semántica**.

**Clase 1 (desastre):**
- *…floods in #Paraguay* → `nearly thousand people affected floods paraguay`  
  - Léxico fuerte: *affected*, *floods*, **topónimo** (*paraguay*).
- *…escape Armageddon* → `vladimir putin issues major warning late escape armageddon`  
  - Palabras catastróficas (no siempre desastre natural, pero señalan severidad).
- *@… burning buildings… riot* → `burning buildings rob riot thats embarrassing ruining nation`  
  - Léxico de **eventos violentos/daño** (*burning*, *buildings*, *riot*).

---

## 5) Calidad de limpieza: hallazgos y mejoras sugeridas

- **4 publicaciones vacías** tras limpieza → recomendación: *drop* o marcar para tratamiento especial (no aportan señal textual).  
- **Mojibake / artefactos Unicode** (p. ej., `û÷`) persisten en algunos textos.
  - *Sugerencia*: añadir una etapa opcional para **filtrar caracteres no alfabéticos** fuera de rangos comunes o normalizar a ASCII cuando el idioma sea inglés:
    - Ej.: `re.sub(r"[^a-z0-9\s\-]", " ", text)` tras NFKC (cuidar no perder topónimos con tildes si hubiera idioma mixto).
- **Números**: se conservó **“911”** y se eliminaron otros números puros.
  - *Sugerencia*: además de “911”, puede valer la pena **conservar números decimales** y patrones de magnitud (*“5.8”*, *“m5.0”*) por su relación con **terremotos**.
  - Regla posible: mantener `r"\d+\.\d+"` (decimales) y tokens con letras+digits (*m5.0, h2o* ya se conservan).

---

## 6) Implicaciones para unigramas/bigramas (inciso 4)

- **Unigramas** útiles (intuición por ejemplos): `wildfire`, `evacuation`, `shelter`, `floods`, `smoke`, `burning`, `evacuate`, `earthquake`, `aftershock`, `landslide`, `tornado`, `explosion`, `derailment`, `casualties`, `emergency`, `fatalities`, `rescue`.  
- **Bigrams** muy recomendables para **contexto**:
  - `forest fire`, `car crash`, `shelter in`, `in place`, `evacuation orders`, `burning buildings`, `state emergency`, `death toll`, `flash flood`, `wildfire smoke`.  
  - También **topónimo + evento**: `paraguay floods`, `alaska wildfire`, etc.
- **Trigrams** puntuales: `shelter in place`, `state of emergency`.  
  - Útiles pero más escasos; probar **bigrams primero** y evaluar mejora.

> Conclusión: sí **vale la pena explorar bigramas** (y unos pocos trigrams clave) para capturar **frases indicadoras** que desambigüen palabras sueltas.

---

## 7) Palabras con riesgo de confusión (discutir en el reporte)

- **Hiperboles cotidianas** (*“near death”*, *“bombed this exam”*, *“my phone exploded”*): pueden **sobrerreaccionar** en unigramas.  
  - Bigrams/trigrams y señales **de contexto real** (lugares, cifras, verbos de reporte: *evacuate, rescue, declare*) ayudan a reducir falsos positivos.
- **Polisemia** (*inundated* = abrumado vs inundado real).  
  - **Topónimos**, **fechas/horas**, **números de víctimas**, y palabras de **autoridad** (*officials, declared, warning, alert*) tienden a correlacionar mejor con eventos reales.

---

## 8) Recomendaciones prácticas para el modelo preliminar

1. **Features**: TF–IDF **unigramas+bigramas** con min_df bajo (p. ej., 2–5) y `max_features` razonable (20k–50k), n-gram range `(1,2)`.  
2. **Clase desbalanceada moderada** (57/43): probar **calibración de clase** (`class_weight='balanced'` en modelos lineales) y **AUC/F1** como métricas.  
3. **Deduplicación opcional**: hay **110 textos duplicados**; considerar *drop* para evitar sesgos en validación.  
4. **Validación**: *stratified split* y baseline con **Logistic Regression** / **Linear SVM**; comparar con Naive Bayes.  
5. **Top tokens por clase**: antes del modelo, obtener **frecuencias por clase** y **nubes de palabras** para documentar (inciso 4 y 5).

---

## 9) Vista previa (sanity check)

Se observa que el **contenido semántico central** se conserva, p. ej.:  
- `deeds reason earthquake allah forgive`  
- `residents asked shelter place …`  
- `people receive wildfires evacuation orders california …`  
- `smoke wildfire … alaska …`  

> Esto confirma que la limpieza **respeta los términos clave** asociados a desastres y reduce ruido social.

---

### Próximo paso
Proceder con **inciso 4**: conteos de frecuencia **por clase** (unigramas y bigramas), discusión de términos útiles y visualizaciones (nube de palabras + histogramas).
